In [45]:
import pandas as pd

In [56]:
df = pd.read_csv('./data/train.csv')

In [57]:
df.columns

Index(['怎么更改花呗手机号码', '我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号', '1'], dtype='object')

In [58]:
df.columns = ['q1', 'q2', 'label']

In [59]:
df.head(5)

,q1,q2,label
0,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0
1,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0
2,如何得知关闭借呗,想永久关闭借呗,0
3,花呗扫码付钱,二维码扫描可以用花呗吗,0
4,花呗逾期后不能分期吗,我这个 逾期后还完了 最低还款 后 能分期吗,0


In [6]:
def single_convert(x, df, df_new):
    q1 = x
    label = 0
    
    df2 = df.sample(frac=0.01)
    
    for item in df2.itertuples():
        q2 = item[2]

        df_new = df_new.append({'q1': q1, 'q2': q2, 'label': 0}, ignore_index = True)
        
    return df_new

    

In [7]:
for i in df.itertuples():
    index = i[0]
    q1 = i[1]
    q2 = i[2]
    label = i[3]
    
    df_new = pd.DataFrame(columns=['q1', 'q2', 'label'])
    
    df_new = single_convert(q1, df, df_new)
    
    df_new.to_csv('./data/enhancement/data_enhancement_%d.csv' % (index), index=None, header=None)
    
    

OSError: [Errno 28] No space left on device

In [54]:
df_new.head(10)

,q1,q2,label
0,0,也开不了花呗，就这样了？完事了,0
1,0,也开不了花呗，就这样了？完事了,0
2,也开不了花呗，就这样了？完事了,也开不了花呗，就这样了？完事了,0
3,也开不了花呗，就这样了？完事了,也开不了花呗，就这样了？完事了,0
4,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0
5,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0
6,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0
7,也开不了花呗，就这样了？完事了,我的条件可以开通花呗借款吗,0
8,也开不了花呗，就这样了？完事了,想永久关闭借呗,0
9,也开不了花呗，就这样了？完事了,二维码扫描可以用花呗吗,0


In [55]:
df_new.to_csv('./data/data_enhancement.csv', index=None, header=None)

In [60]:
import jieba

In [61]:
def empty_filter(word):
    if '' == word:
        return False
    if len(word) == 1:
        return False
    return True

def jieba_cut(sentence):
    word_arr = jieba.cut(sentence.strip(), cut_all=False)
    word_arr = list(word_arr)
    word_arr = filter(empty_filter, word_arr)
    return ','.join(list(word_arr))

def get_max_word(sentence):
    word_arr = jieba_cut(sentence=sentence)
    length = -1
    word = None
    for item in word_arr:
        if len(item) > length:
            length = len(item)
            word = item
    return word


In [62]:
df['q3'] = df['q1'].apply(lambda x: jieba_cut(x))

In [63]:
df.head(5)

,q1,q2,label,q3
0,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,"不了,这样,完事"
1,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0,"冻结,以后,开通"
2,如何得知关闭借呗,想永久关闭借呗,0,"如何,得知,关闭"
3,花呗扫码付钱,二维码扫描可以用花呗吗,0,"扫码,付钱"
4,花呗逾期后不能分期吗,我这个 逾期后还完了 最低还款 后 能分期吗,0,"逾期,不能,分期"


In [64]:
df = (df.set_index(['q1','q2','label'])['q3']
       .str.split(',', expand=True)
       .stack()
       .reset_index(level=3, drop=True)
       .reset_index())

In [65]:
df.head(5)

,q1,q2,label,0
0,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,不了
1,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,这样
2,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,完事
3,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0,冻结
4,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0,以后


In [66]:
df.label = 0
df.columns = ['q1', 'q2', 'label', 'q3']

In [67]:
df.head(5)

,q1,q2,label,q3
0,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,不了
1,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,这样
2,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,完事
3,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0,冻结
4,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0,以后


In [69]:
df.to_csv('./data/data_word_enhancement.csv', index=None, header=None, columns=['q3', 'q2', 'label'])